# Matplotlib & Pandas Graphing Guide (All-in-One, English Labels)
This notebook combines the **Matplotlib basics** and the **Pandas + CSV/Excel real-data module** in one place, with all chart titles and axis labels in **English** to avoid font issues.

**Learning path**: (1) Hand-made lists → (2) Functions + `append` → (3) NumPy vectorization → (4) Read CSV/Excel → (5) Aggregation & visualization → (6) Missing values → (7) Finishing touches.


## Table of Contents
- Basics
  - 0. Setup
  - Step 1: Hand-made lists
  - Step 2: Functions + `append`
  - Step 3: NumPy vectorization
  - Advanced: Subplots & annotations
- Real Data
  - Step A: DataFrame basics
  - Step B: CSV (Sales)
  - Step C: Excel (Sensors)
  - Step D: Missing values & cleaning
  - Step E: Finishing customization
- Summary


---
# Basics: Plotting with Matplotlib
## 0. Setup (imports & display)
We import Matplotlib / NumPy / Pandas and set a neutral style. Japanese-capable fonts are optional; all labels below are in English.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')
from matplotlib import rcParams
# Keep minus sign correct
rcParams['axes.unicode_minus'] = False
print('pandas:', pd.__version__)
print('Matplotlib:', plt.matplotlib.__version__)
print('NumPy:', np.__version__)


## Step 1: Plot from hand-made lists
Create line, scatter, and bar charts from simple Python lists.

In [ ]:
x = [0, 1, 2, 3, 4, 5]
y = [0, 1, 4, 9, 16, 25]  # y = x^2

fig, axes = plt.subplots(1, 3, figsize=(13, 3.5))
axes[0].plot(x, y, marker='o', color='#2a9d8f')
axes[0].set_title('Line (y = x^2)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[1].scatter(x, y, color='#e76f51')
axes[1].set_title('Scatter (y = x^2)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[2].bar(x, y, color='#457b9d')
axes[2].set_title('Bar (y = x^2)')
axes[2].set_xlabel('x')
axes[2].set_ylabel('y')
fig.suptitle('Step 1: Plotting from lists', fontsize=14)
plt.tight_layout(); plt.show()


### Practice (Step 1)
1. Change `y` to `x^3` and redraw.
2. Tune `color`, `marker`, and `linewidth`.

## Step 2: Use functions + `append` to build lists
Compute values using functions and append them in a loop.

In [ ]:
def f(x):
    return math.sin(x)
def g(x):
    return math.sin(x) + 0.3 * x
x2, y_f, y_g = [], [], []
cur = 0.0
while cur <= 10.0 + 1e-9:
    x2.append(cur); y_f.append(f(cur)); y_g.append(g(cur)); cur += 0.2
plt.figure(figsize=(6,4))
plt.plot(x2, y_f, label='f(x)=sin(x)', color='#264653')
plt.plot(x2, y_g, label='g(x)=sin(x)+0.3x', color='#e9c46a')
plt.title('Step 2: Function + append')
plt.xlabel('x'); plt.ylabel('value'); plt.legend(); plt.tight_layout(); plt.show()


### Practice (Step 2)
1. Try `g(x) = sin(x) + 0.1 * x^2`.
2. Change the step to `0.1` or `0.5` and compare smoothness.

## Step 3: NumPy vectorization (no loops)
Generate arrays and compute values in bulk with NumPy.

In [ ]:
x3 = np.arange(0, 10.0 + 1e-12, 0.01)
y3_f = np.sin(x3)
y3_g = np.sin(x3) + 0.3 * x3
plt.figure(figsize=(6,4))
plt.plot(x3, y3_f, label='f(x)=sin(x)', color='#2a9d8f')
plt.plot(x3, y3_g, label='g(x)=sin(x)+0.3x', color='#f4a261')
plt.title('Step 3: NumPy vectorization')
plt.xlabel('x'); plt.ylabel('value'); plt.legend(); plt.tight_layout(); plt.show()


### Practice (Step 3)
1. Use `np.linspace(start, stop, num)` and compare with `arange`.
2. Plot `y = exp(-x/3) * sin(3x)`.
3. Plot the difference `y3_g - y3_f`.

## Advanced: Subplots & annotations
Arrange multiple charts and add annotations.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 6), sharex=True)
x = np.linspace(0, 2*np.pi, 200)
axs[0,0].plot(x, np.sin(x), color='#3a86ff'); axs[0,0].set_title('sin(x)')
axs[0,1].plot(x, np.cos(x), color='#ff006e'); axs[0,1].set_title('cos(x)')
axs[1,0].plot(x, np.tan(x), color='#8338ec'); axs[1,0].set_title('tan(x)'); axs[1,0].set_ylim(-3,3)
y = np.exp(-x/2)*np.sin(3*x)
axs[1,1].plot(x, y, color='#fb5607'); axs[1,1].set_title('exp(-x/2) * sin(3x)')
peak_x = x[np.argmax(y)]; peak_y = y.max()
axs[1,1].annotate('peak', xy=(peak_x, peak_y), xytext=(peak_x+0.5, peak_y+0.2), arrowprops=dict(arrowstyle='->', color='gray'))
for ax in axs.flat: ax.set_xlabel('x'); ax.set_ylabel('y')
fig.suptitle('Advanced: Subplots & annotation', fontsize=14)
plt.tight_layout(); plt.show()


---
# Real Data: Pandas × Matplotlib (CSV/Excel)
If sample files are missing, the next cell will generate synthetic data for class use. All labels will be in English; data column names may be in Japanese but we create English aliases for plotting.

In [ ]:
import os
from datetime import datetime, timedelta
np.random.seed(42)
if not os.path.exists('sales_sample.csv') or not os.path.exists('sensor_sample.xlsx'):
    # Sales CSV
    dates = pd.date_range('2024-01-01', periods=90, freq='D')
    categories = ['A食品', 'B飲料', 'C雑貨']
    rows = []
    for d in dates:
        for c in categories:
            base = {'A食品': 120, 'B飲料': 80, 'C雑貨': 60}[c]
            season = 10*np.sin(2*np.pi*(d.dayofyear/365.0))
            noise = np.random.normal(0, 15)
            qty = max(0, int(base + season + noise))
            price = {'A食品': 250, 'B飲料': 180, 'C雑貨': 500}[c]
            rows.append({'日付': d, 'カテゴリ': c, '数量': qty, '単価': price, '金額': qty*price})
    sales_df = pd.DataFrame(rows)
    sales_df.to_csv('sales_sample.csv', index=False, encoding='utf-8')
    # Sensor Excel
    t0 = datetime(2024, 2, 1)
    times = [t0 + timedelta(minutes=15*i) for i in range(24*4*5)]
    sensor_rows = []
    for t in times:
        temp = 18 + 6*np.sin(2*np.pi*(t.hour/24)) + np.random.normal(0, 0.8)
        hum = 55 + 12*np.cos(2*np.pi*(t.hour/24)) + np.random.normal(0, 2.0)
        if np.random.rand() < 0.02: temp = np.nan
        if np.random.rand() < 0.02: hum = np.nan
        sensor_rows.append({'時刻': t, '温度(℃)': round(temp,2), '湿度(%)': round(hum,2)})
    sensor_df = pd.DataFrame(sensor_rows)
    sensor_df.to_excel('sensor_sample.xlsx', index=False, engine='openpyxl')
print('OK: sample files ready')


## Step A: DataFrame basics (hand-made data)
Compare `DataFrame.plot` and pure Matplotlib.

In [ ]:
df = pd.DataFrame({
    'month': ['Jan','Feb','Mar','Apr','May','Jun'],
    'sales_A': [12, 14, 11, 18, 17, 20],
    'sales_B': [9, 10, 13, 15, 16, 19]
})
display(df)
ax = df.set_index('month').plot(kind='bar', figsize=(6,4), color=['#2a9d8f','#e76f51'])
ax.set_title('Category Sales (hand-made)'); ax.set_xlabel('month'); ax.set_ylabel('amount')
plt.tight_layout(); plt.show()
x = np.arange(len(df)); width = 0.35
fig, ax = plt.subplots(figsize=(6,4))
ax.bar(x - width/2, df['sales_A'], width, label='sales_A', color='#457b9d')
ax.bar(x + width/2, df['sales_B'], width, label='sales_B', color='#f4a261')
ax.set_xticks(x); ax.set_xticklabels(df['month'])
ax.set_title('Category Sales (Matplotlib)'); ax.set_xlabel('month'); ax.set_ylabel('amount'); ax.legend(); plt.tight_layout(); plt.show()


### Practice (A)
- Switch to a line chart (`kind='line'`) and adjust legend/colors.
- Run `df.describe()` to check basic statistics.

## Step B: Read CSV (Sales)
Read `sales_sample.csv` with `parse_dates=['日付']`. Create **English aliases** to keep plot labels in English.

In [ ]:
sales = pd.read_csv('sales_sample.csv', encoding='utf-8', parse_dates=['日付'])
# Create English aliases for convenience
sales_en = sales.rename(columns={'日付':'date','カテゴリ':'category','数量':'qty','単価':'unit_price','金額':'amount'})
sales_en.head()


In [ ]:
daily = sales_en.groupby(['date','category'], as_index=False)['amount'].sum()
pivot = daily.pivot(index='date', columns='category', values='amount')
ax = pivot.plot(figsize=(7,4), title='Daily Sales by Category', ylabel='amount')
ax.set_xlabel('date'); plt.tight_layout(); plt.show()
monthly = sales_en.groupby([sales_en['date'].dt.to_period('M'),'category'])['amount'].sum().reset_index()
monthly['month'] = monthly['date'].astype(str)
ax = monthly.pivot(index='month', columns='category', values='amount').plot(kind='bar', figsize=(8,4), title='Monthly Sales (Total)')
ax.set_xlabel('month'); ax.set_ylabel('amount'); plt.tight_layout(); plt.show()
ma7 = pivot.rolling(window=7, min_periods=1).mean()
ax = ma7.plot(figsize=(7,4), title='Daily Sales (7-day MA)', ylabel='amount')
ax.set_xlabel('date'); plt.tight_layout(); plt.show()


### Practice (B)
- Analyze **average unit price** (`amount/qty`) and plot it.
- Group by weekday using `sales_en['date'].dt.day_name()` and make a bar chart.

## Step C: Read Excel (Sensors)
Read `sensor_sample.xlsx` (English labels on plots). Create English aliases for columns.

In [ ]:
sensor = pd.read_excel('sensor_sample.xlsx', engine='openpyxl', parse_dates=['時刻'])
sensor_en = sensor.rename(columns={'時刻':'time','温度(℃)':'temp_C','湿度(%)':'humidity_pct'})
sensor_en.head()


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8,6), sharex=True)
ax[0].plot(sensor_en['time'], sensor_en['temp_C'], color='#3a86ff'); ax[0].set_title('Temperature (15-min)'); ax[0].set_ylabel('°C')
ax[1].plot(sensor_en['time'], sensor_en['humidity_pct'], color='#ff006e'); ax[1].set_title('Humidity (15-min)'); ax[1].set_xlabel('time'); ax[1].set_ylabel('%')
plt.tight_layout(); plt.show()
sensor_idx = sensor_en.set_index('time')
hourly = sensor_idx.resample('1H').mean()
ax = hourly.plot(figsize=(8,4), title='Temperature & Humidity (1-hour mean)')
ax.set_xlabel('time'); ax.set_ylabel('value'); plt.tight_layout(); plt.show()
plt.figure(figsize=(6,4))
plt.scatter(sensor_en['temp_C'], sensor_en['humidity_pct'], alpha=0.5, color='#8338ec')
plt.title('Temperature vs Humidity (scatter)'); plt.xlabel('temperature (°C)'); plt.ylabel('humidity (%)'); plt.tight_layout(); plt.show()
sensor_en[['temp_C','humidity_pct']].hist(figsize=(8,3), bins=20)
plt.suptitle('Distributions: Temperature / Humidity'); plt.tight_layout(); plt.show()


### Practice (C)
- Smooth with a 24-hour moving average: `rolling('24H')`.
- Fit a regression line (`np.polyfit`) on the scatter and overlay it.

## Step D: Missing values & cleaning
Handle NaNs via `dropna` / `fillna` / `interpolate`.

In [ ]:
missing_rate = sensor_en.isna().mean()*100
print('Missing rate (%):
', missing_rate)
sensor_clean = sensor_en.set_index('time').interpolate(method='time').reset_index()
fig, ax = plt.subplots(1,2, figsize=(10,3))
ax[0].plot(sensor_en['time'], sensor_en['temp_C'], color='#3a86ff'); ax[0].set_title('Temp (before)')
ax[1].plot(sensor_clean['time'], sensor_clean['temp_C'], color='#3a86ff'); ax[1].set_title('Temp (after interpolation)')
for a in ax: a.set_xlabel('time'); a.set_ylabel('°C')
plt.tight_layout(); plt.show()


### Practice (D)
- Try `fillna(method='ffill')` and `bfill` vs interpolation.
- Drop rows with many missing values and re-plot.

## Step E: Finishing customization
Legend position, line width, colors, annotations, and saving images (`plt.savefig`).

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(hourly.index, hourly['temp_C'], label='temperature (°C)', color='#1f77b4', linewidth=2)
ax.set_title('Finishing: annotation & save'); ax.set_xlabel('time'); ax.set_ylabel('°C')
pk_idx = hourly['temp_C'].idxmax(); pk_val = hourly['temp_C'].max()
ax.annotate('peak', xy=(pk_idx, pk_val), xytext=(pk_idx, pk_val+1), arrowprops=dict(arrowstyle='->', color='gray'))
ax.legend(loc='upper left'); plt.tight_layout(); plt.savefig('sensor_hourly_temperature_en.png', dpi=150); plt.show()
print('Saved file: sensor_hourly_temperature_en.png')


---
# Summary
- **Basics**: lists → functions → NumPy vectorization with Matplotlib.
- **Real Data**: `read_csv`/`read_excel`, `groupby`/`pivot`/`resample`, cleaning & saving.
All labels are in English to avoid font issues across environments.
